In [ ]:
from math import sqrt
from operator import itemgetter

import numpy as np
from scipy.linalg import svd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn import metrics


In [ ]:
class MFMPC:
    def __init__(self, records_train, records_test):
        records = np.vstack([records_train, records_test])
        self.n = len(np.unique(np.sort(records[:, 0])))
        self.m = len(np.unique(np.sort(records[:, 1])))

        # split explict and implicit feedback
        np.random.seed(666)
        indexes = np.random.permutation(len(records_train))
        records_train_ex = records_train[indexes[:len(indexes) // 2]]
        records_train_im = records_train[indexes[len(indexes) // 2:]]

        # Initial R
        self.R = np.zeros([self.n, self.m], dtype=np.int32)

        for record in records_train_ex:
            self.R[record[0], record[1]] = record[2]
            
        # Initial O
        self.O = np.zeros([self.n, self.m], dtype=np.int32)
        
        for record in records_train_im:
            self.O[record[0], record[1]] = 1

        # Initial indicator
        y = np.where(self.R, 1, 0)
        y_user = np.sum(y, axis=1)
        y_item = np.sum(y, axis=0)

        # Global average of rating
        self.r = np.sum(self.R) / np.sum(y)

        # average rating of user
        self.r_u = np.where(y_user,
                            np.sum(self.R, axis=1) / y_user,
                            self.r)

        # average rating of item
        self.r_i = np.where(y_item,
                            np.sum(self.R, axis=0) / y_item,
                            self.r)

        # bias of user
        self.b_u = np.where(y_user,
                            np.sum(y * (self.R - self.r_i), axis=1) / y_user,
                            0)

        # bias of item
        self.b_i = np.where(y_item,
                            np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,
                            0)
        
    def gradient_descent(self, n_iter=10):

        alpha = 0.01
        d = 20
        
        # Initialize
        self.U = (np.random.randint(0, 1, size=(self.n, d)) - 0.5) * 0.01
        self.V = (np.random.randint(0, 1, size=(self.m, d)) - 0.5) * 0.01
        self.W = (np.random.randint(0, 1, size=(self.m, d)) - 0.5) * 0.01
        self.mu = self.r
 
        
        eta = 0.05
        
        def update(user, item, r):
            # Calculate neighbours
            items_im = np.nonzero(self.O[user])[0]
            U_tilde = np.sum(self.W[items_im], axis=0) / sqrt(len(items_im))
            
            # update
            e = r - (self.mu + self.b_u[user] + self.b_i[item] + (self.U[user] + U_tilde).dot(self.V[item]))
            d_mu = -e
            d_b_u = -e + alpha * self.b_u[user]
            d_b_i = -e + alpha * self.b_i[item]
            d_U =  -e * self.V[item] + alpha * self.U[user]
            d_V = -e * (self.U[user] + U_tilde) + alpha * self.V[item]
            d_W = -e / sqrt(len(items_im)) * self.V[item] + alpha * self.W[items_im]
            self.mu -= eta * d_mu
            self.b_u[user] -= eta * d_b_u
            self.b_i[item] -= eta * d_b_i
            self.U[user, :] -= eta * d_U
            self.V[item, :] -= eta * d_V
            self.W[items_im, :] -= eta * d_W
        
        for cur_iter in range(n_iter):
            print(cur_iter)
            ratings = np.where(self.R != 0)
            num = len(ratings[0])
            indexes = np.random.permutation(num)
            users = ratings[0][indexes]
            items = ratings[1][indexes]

            for i in range(num):
                user = users[i]
                item = items[i]
                update(user, item, self.R[user, item])

            eta = eta * 0.9
            ratings_predict = self.performance()
            print(score(np.clip(ratings_predict, 1, 5), ratings_test))

        return
    
    def performance(self):
        ratings_predict = np.empty(len(records_test))
        U_tilde = np.zeros([self.m ,20])
        for user in range(self.n):
            items_im = np.nonzero(self.O[user])[0]
            if len(items_im):
                U_tilde[user] = np.sum(self.W[items_im], axis=0) / sqrt(len(items_im))
        
        for i in range(len(records_test)):
            user = records_test[i][0]
            item = records_test[i][1]
            rating_predict = self.mu + self.b_u[user] + self.b_i[item] + (self.U[user] + U_tilde[user]).dot(self.V[item])
            ratings_predict[i] = rating_predict
        return ratings_predict

In [ ]:
def score(ratings_test, ratings_predict):
    return [round(sqrt(metrics.mean_squared_error(ratings_test, ratings_predict)), 4),
            round(metrics.mean_absolute_error(ratings_test, ratings_predict), 4)]


In [ ]:
# Load the records
records_train = np.loadtxt('../data/ml-100k/ua.base', dtype=np.int32)
records_test = np.loadtxt('../data/ml-100k/ua.test', dtype=np.int32)

# Preprocess
records_train[:, :2] -= 1
records_test[:, :2] -= 1
ratings_test = records_test[:, 2]
records = np.vstack([records_train, records_test])


In [ ]:
svdpp = SVDpp(records_train, records_test)
svdpp.gradient_descent(20)

In [ ]:
tt = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])

In [ ]:
print(tt[:2])
print(tt[2:])


In [ ]:
np.random.seed(666)
np.random.permutation(6)



In [ ]:
ttt = np.empty(1)

ttt